# 本文主要涉及到3个函数：
torch.save: 使用Python的pickle实用程序将对象进行序列化，然后将序列化的对象保存到disk，可以保存各种对象,包括模型、张量和字典等。
torch.load: 使用pickle unpickle工具将pickle的对象文件反序列化为内存。
torch.nn.Module.load_state_dict: 用反序列化的state_dict来加载模型参数。
# 参考资料
https://blog.csdn.net/wangkaidehao/article/details/104296025

# 1. 读取tensor
## 1.1 单个张量

In [1]:
import torch

# create tensor
x = torch.tensor([3.,4.])
# save model as file  
torch.save(x, 'x.pt')

# load file
x1 = torch.load('x.pt')
print(x1)

/home/ty6232/miniconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([3., 4.])


## 1.2 张量列表和张量词典

In [2]:
# 创建一个tensor列表 4列2行
y = torch.ones((4,2))
# save load xy两个参数
torch.save([x,y],'xy.pt')
# 添加参数描述 构成tensor词典
torch.save({'x':x, 'y':y}, 'xy_dict.pt')
xy = torch.load('xy.pt')
xy_dict = torch.load('xy_dict.pt')
print(xy)
print(xy_dict)

[tensor([3., 4.]), tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])]
{'x': tensor([3., 4.]), 'y': tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])}


# 2 保存和加载模型
## 2.1 state_dict
state_dict是一个从每一个层的名称映射到这个层的参数Tesnor的字典对象。

注意，只有具有可学习参数的层(卷积层、线性层等)和注册缓存(batchnorm’s running_mean)才有state_dict中的条目。优化器(torch.optim)也有一个state_dict，其中包含关于优化器状态以及所使用的超参数的信息。

In [3]:
from torch import nn
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # 创建hidden参数
        self.hidden = nn.Linear(3, 2) 
        # Relu激活参数
        self.act = nn.ReLU()
        # 输出层
        self.output = nn.Linear(2, 1)

    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

net = MLP()
# 输出可学习的参数 hiden.weight hidden.biad output.weight output.bias
print(net.state_dict()) 
print('\n',net.state_dict()['output.weight'])

# 优化器，SDG随机下降梯度
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
print(optimizer.state_dict())

OrderedDict([('hidden.weight', tensor([[ 0.0562, -0.1416,  0.1328],
        [ 0.2583, -0.0406,  0.2048]])), ('hidden.bias', tensor([0.1066, 0.3095])), ('output.weight', tensor([[0.0007, 0.4009]])), ('output.bias', tensor([-0.6252]))])

 tensor([[0.0007, 0.4009]])
{'state': {}, 'param_groups': [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0, 1, 2, 3]}]}


## 2.2 保存和加载
PyTorch中保存和加载训练模型有两种常见的方法:

仅保存和加载模型参数(state_dict)；
保存和加载整个模型。

### 2.2.1 保存和加载state_dict(推荐方式)

In [4]:
# 模型序列化存储
torch.save(net.state_dict(), 'net_state_dict.pt')## 后缀名一般写为: .pt或.pth
net1 = MLP()

# load反序列化
net1.load_state_dict(torch.load('net_state_dict.pt'))

<All keys matched successfully>

load_state_dict() 接受一个词典对象，而不是一个指向对象的路径。

## 2.2.2 保存和读写整个模型

In [ ]:
torch.save(net, 'net.pt')
net2 = torch.load('net.pt')